# Graph Statistics

In this notebook, I will show how to obtain basic graph statistics (number of nodes, density, etc.) from the graphs stored in our filespec.

## Load the Graphs

In [94]:
from c_elegans.witvilet2020.load_wit import load_wit
from c_elegans.worm_wiring.load_worm import load_worm
import networkx as nx
from networkx.algorithms.components import number_strongly_connected_components as ncc_strong
from networkx.algorithms.components import number_weakly_connected_components as ncc_weak
import matplotlib.pyplot as plt
import numpy as np

worm_graphs = load_worm()
wit_graphs = load_wit()
worm_connectomes = worm_graphs
for g in worm_graphs:
    print("g.graph=", g.graph)

g.graph= {'Title': 'L4 nerve ring neighbors'}
g.graph= {'Title': 'Adult nerve ring neighbors'}
g.graph= {'Title': 'Cell Class Adjacency Matrices  These are whole-animal connectivity matrices based on the cell classes listed in Supplementary Information 6.  The data are the same (number of EM serial sections of connectivity) as that in Supplementary Information 5, except given by cell class (or larger combined group) instead of by individual cell.  Connectivity between classes is the sum of the connectivity of the class members given in Supplementary Information 5.  Data in these tables is used for the Figures in Cook et al.\nCorrections, July 2020  An effort has been made to remove all errors and inconsistencies within and between the matrices.  There were previously a number of them.', 'Sex': 'Hermaphrodite', 'Synapse Type': 'Symmetric Gap Junction'}
g.graph= {'Title': 'Cell Class Adjacency Matrices  These are whole-animal connectivity matrices based on the cell classes listed in Supp

## Obtain Graph Statistics and Metadata

In [92]:
lcc_size = [] #Instead of number of nodes -- number of nodes in largest weakly connected component
density = []
num_strong = []
num_weak = []
synapse_type = []
sex = []
in_degree = []
out_degree = []
graph_origin = []

for g in worm_connectomes:
    graph_origin.append("worm")
    lcc_size.append(len(max(nx.weakly_connected_components(g))))
    density.append(nx.density(g))
    num_strong.append(ncc_strong(g))
    num_weak.append(ncc_weak(g))
    try:
        synapse_type.append(g.graph['Synapse Type'])
    except KeyError:
        synapse_type.append("Unknown")
    try:
        sex.append(g.graph['Sex'])
    except KeyError:
        sex.append("Unknown")
    in_degree.append([d for _, d in g.in_degree()])
    out_degree.append([d for _, d in g.out_degree()])
    
for graph in wit_graphs:
    chemical_edges = []
    electrical_edges = []
    for *edge, attr in graph.edges(data=True, keys=True):
        if attr['synapse_type'] == "chemical":
            chemical_edges.append((*edge,))
        elif attr['synapse_type'] == "electrical":
            electrical_edges.append((*edge,))
    g_chemical = graph.edge_subgraph(chemical_edges)
    g_electrical = graph.edge_subgraph(electrical_edges)
    for g, syn_type in [(g_chemical, "Chemical"), (g_electrical, "Total Gap Junction")]:
        graph_origin.append("wit")
        lcc_size.append(len(max(nx.weakly_connected_components(g))))
        density.append(nx.density(g))
        num_strong.append(ncc_strong(g))
        num_weak.append(ncc_weak(g))
        synapse_type.append(syn_type)
        sex.append(g.graph['Sex'])
        in_degree.append([d for _, d in g.in_degree()])
        out_degree.append([d for _, d in g.out_degree()])

sex = np.array(sex)
lcc_size = np.array(lcc_size)
density = np.array(density)
num_strong = np.array(num_strong)
num_weak = np.array(num_weak)
graph_origin = np.array(graph_origin)
in_degree = [np.array(ins) for ins in in_degree]
out_degree = [np.array(outs) for outs in out_degree]
synapse_type = np.array(synapse_type)

## Visualize 

In [93]:
print(synapse_type[graph_origin=="worm"])

syntypes = ["Chemical", "Symmetric Gap Junction", "Asymmetric Gap Junction", "Total Gap Junction"]
syntype_pos = [0, 1.25, 2., 2.75]
syntype2pos = {syntype:i for i, syntype in zip(syntype_pos, syntypes)}
syn_pos = np.array([syntype2pos[syntype] for syntype in synapse_type])
syn_pos += 0.25*(np.random.rand(syn_pos.shape[0]) - 0.5) #adding jitter
sexes = ["Male", "Hermaphrodite"]
sex2color = {"Male":'b', "Hermaphrodite":'purple'}
sex_colors = [sex2color[isex] for isex in sex]

def vis_stat(stat, stat_name):
    fig_max = np.max(stat)
    fig = plt.figure()
    ax = plt.gca()
    plt.hold=True
    for isex in sexes:
        plt.scatter(syn_pos[np.logical_and(sex==isex, graph_origin=="worm")],
                    stat[np.logical_and(sex==isex, graph_origin=="worm")],
                    c=sex2color[isex],
                    label=isex+", worm wiring",
                    marker='o')
        plt.scatter(syn_pos[np.logical_and(sex==isex, graph_origin=="wit")],
                    stat[np.logical_and(sex==isex, graph_origin=="wit")],
                    c=sex2color[isex],
                    label=isex+", witvilet",
                    marker='s')
    plt.text(syntype_pos[0], 1.1*fig_max, "Chemical", ha="center")
    plt.text(syntype_pos[2], 1.1*fig_max, "Electrical", ha="center")
    plt.hold=False
    plt.ylabel(stat_name)
    ax.xaxis.set_ticks(syntype_pos)
    ax.xaxis.set_ticklabels(syntypes, rotation=90)
    fill_width = 2
    ax.axvspan(syntype_pos[0]-fill_width/2, syntype_pos[0]+fill_width/2, color="yellow", alpha=0.1)
    ax.axvspan(syntype_pos[2]-fill_width/2, syntype_pos[2]+fill_width/2, color="red", alpha=0.1)
    #plt.legend(loc='best', ncol=3)
    return fig
    
fig = vis_stat(lcc_size, "Number of Nodes in the Largest Weakly Connected Component")
fig.savefig("num_nodes.png", bbox_inches="tight")
fig = vis_stat(density, "Density")
fig.savefig("density.png", bbox_inches="tight")
fig = vis_stat(num_strong, "Number of Strongly Connected Components")
fig.savefig("num_strong.png", bbox_inches="tight")
fig = vis_stat(num_weak, "Number of Weakly Connected Components")
fig.savefig("num_weak.png", bbox_inches="tight")

['Unknown' 'Unknown' 'Symmetric Gap Junction' 'Asymmetric Gap Junction'
 'Symmetric Gap Junction' 'Symmetric Gap Junction'
 'Asymmetric Gap Junction' 'Symmetric Gap Junction' 'Gap Junction'
 'Gap Junction' 'Gap Junction' 'Gap Junction' 'Symmetric Gap Junction'
 'Asymmetric Gap Junction' 'Symmetric Gap Junction'
 'Symmetric Gap Junction' 'Asymmetric Gap Junction'
 'Symmetric Gap Junction']


KeyError: 'Unknown'

Witvilet-specific graphs

In [77]:
ages = []
for g in wit_graphs:
    ages.append(g.graph['age'])

In [80]:
def vis_stat(stat, stat_name):
    fig_max = np.max(stat)
    fig = plt.figure()
    ax = plt.gca()
    plt.hold=True
    for syntype in syntypes:
        plt.scatter(ages, graph_origin=="worm")],
                    stat[np.logical_and(sex==isex, graph_origin=="worm")],
                    c=sex2color[isex],)
    plt.text(syntype_pos[0], 1.1*fig_max, "Chemical", ha="center")
    plt.text(syntype_pos[2], 1.1*fig_max, "Electrical", ha="center")
    plt.hold=False
    plt.ylabel(stat_name)
    ax.xaxis.set_ticks(syntype_pos)
    ax.xaxis.set_ticklabels(syntypes, rotation=90)
    fill_width = 2
    ax.axvspan(syntype_pos[0]-fill_width/2, syntype_pos[0]+fill_width/2, color="yellow", alpha=0.1)
    ax.axvspan(syntype_pos[2]-fill_width/2, syntype_pos[2]+fill_width/2, color="red", alpha=0.1)
    #plt.legend(loc='best', ncol=3)
    return fig
    
fig = vis_stat(lcc_size, "Number of Nodes in the Largest Weakly Connected Component")
fig.savefig("num_nodes.png", bbox_inches="tight")
fig = vis_stat(density, "Density")
fig.savefig("density.png", bbox_inches="tight")
fig = vis_stat(num_strong, "Number of Strongly Connected Components")
fig.savefig("num_strong.png", bbox_inches="tight")
fig = vis_stat(num_weak, "Number of Weakly Connected Components")
fig.savefig("num_weak.png", bbox_inches="tight")

lcc_size= [446 460 460 559  57  57 221 163 219 154 187  28 193  86 198   2 203 111
 210 151 216 139]
synapse_type= ['Symmetric Gap Junction', 'Asymmetric Gap Junction', 'Symmetric Gap Junction', 'Symmetric Gap Junction', 'Asymmetric Gap Junction', 'Symmetric Gap Junction', 'Chemical', 'Total Gap Junction', 'Chemical', 'Total Gap Junction', 'Chemical', 'Total Gap Junction', 'Chemical', 'Total Gap Junction', 'Chemical', 'Total Gap Junction', 'Chemical', 'Total Gap Junction', 'Chemical', 'Total Gap Junction', 'Chemical', 'Total Gap Junction']
